In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 21.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional

def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

    def vector_query(
        query: str,
        page_numbers: Optional[List[str]] = None
    ) -> str:
        """Use to answer questions over the MetaGPT paper.

        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )
        response = query_engine.query(query)
        return response


    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=vector_query
    )

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            "Use ONLY IF you want to get a holistic summary of MetaGPT. "
            "Do NOT use if you have specific questions over MetaGPT."
        ),
    )

    return vector_query_tool, summary_tool

In [ ]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-06-01 22:40:13--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  38.5MB/s    in 0.4s    

2024-06-01 22:40:13 (38.5 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [ ]:
from getpass import getpass
import os

GROQ_API_TOKEN = getpass()

··········


In [ ]:
from llama_index.llms.groq import Groq

from llama_index.core import Settings, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#llm = Groq(model="llama3-8b-8192", api_key=GROQ_API_TOKEN)
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_TOKEN)
Settings.llm = llm

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

In [ ]:
from llama_index.core.agent import ReActAgent


query_engine_tools = [vector_tool, summary_tool]

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [ ]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, and how they communicate with each other."
)
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'agent roles in MetaGPT', 'page_numbers': None}
Observation: In MetaGPT, there are five defined agent roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer.
Thought: I need more information about the communication between these agent roles.
Action: vector_tool_metagpt
Action Input: {'query': 'communication between agent roles in MetaGPT'}
Observation: In MetaGPT, communication between agent roles is facilitated through a shared message pool, where agents publish structured messages and can subscribe to relevant messages based on their role profiles. This enables agents to obtain directional information from other roles and public information from the environment, enhancing communication efficiency.
Thought: I have enough information to answer the question without using any more tools.
Answer: In MetaGPT, 

In [ ]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 4
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-01
last_modified_date: 2024-06-01

Preprint
Figure 2: An example of the communication protocol (left) and iterative programming with exe-
cutable feedback (right). Left: Agents use a shared message pool to publish structured messages.
They can also subscribe to relevant messages based on their profiles. Right : After generating the
initial code, the Engineer agent runs and checks for errors. If errors occur, the agent checks past
messages stored in memory and compares them with the PRD, system design, and code files.
3 M ETAGPT: A M ETA-PROGRAMMING FRAMEWORK
MetaGPT is a meta-programming framework for LLM-based multi-agent systems. Sec. 3.1 pro-
vides an explanation of role specialization, workflow and structured communication in this frame-
work, and illustrates how to organize a multi-agent system within the context of SOPs. Sec. 3.2
presents a communicati

In [ ]:
response = agent.query(
    "Tell me about the evaluation datasets used."
)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'evaluation datasets', 'page_numbers': None}
Observation: The SoftwareDev dataset is mentioned, which includes 70 diverse software development tasks.
Thought: I need more information about the evaluation datasets used in MetaGPT.
Action: vector_tool_metagpt
Action Input: {'query': 'evaluation datasets'}
Observation: The SoftwareDev dataset is mentioned, which includes 70 diverse software development tasks.
Thought: I need more information about the evaluation datasets used in MetaGPT.
Action: vector_tool_metagpt
Action Input: {'query': 'evaluation datasets besides SoftwareDev'}
Observation: HumanEval and MBPP.
Thought: I have enough information to answer the question without using any more tools.
Answer: The evaluation datasets used in MetaGPT include SoftwareDev, HumanEval, and MBPP.


In [ ]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 23
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-01
last_modified_date: 2024-06-01

Preprint
Figure 10: The “Drawing App” generated by MetaGPT.
C E XPERIMENTS
C.1 D ETAILS OF THE SOFTWARE DEVDATASET
The SoftwareDev dataset includes 70 diverse software development tasks. Table 8 displays the
names and detailed prompts of 11 tasks within the dataset. Note that the first seven tasks listed are
used in the main experiments of this paper.
C.2 A DDITIONAL RESULTS
Quantitative results of MetaGPT As shown in Table 4, MetaGPT achieves an average score
of 3.9, surpassing ChatDev’s score of 2.1 Zhao et al. (2023), which is based on the Chat chain.
Compare the scores of general intelligent algorithms, including AutoGPT Torantulino et al. (2023),
which all score 1.0, failing to generate executable code. We observe that the generated code is often
short, lacks comprehensive logic, and tends to fail to handle cross-file

In [ ]:
# use agent.chat instead of agent.query to pass conversational history automatically to answer follow up questions
response = agent.chat("Tell me the results over one of the above datasets.")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'What are the results on the SoftwareDev dataset?', 'page_numbers': None}
Observation: The results of MetaGPT on the SoftwareDev dataset show an average score of 3.9, surpassing ChatDev's score of 2.1.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The results of MetaGPT on the SoftwareDev dataset show an average score of 3.9, surpassing ChatDev's score of 2.1.


In [ ]:
# use agent.chat instead of agent.query to pass conversational history automatically to answer follow up questions
response = agent.chat("Tell me more.")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'What are the results of MetaGPT on the SoftwareDev dataset?', 'page_numbers': None}
Observation: The results of MetaGPT on the SoftwareDev dataset show that it achieves an average score of 3.9, surpassing ChatDev's score of 2.1. Additionally, MetaGPT outperforms ChatDev on nearly all metrics, including executability, code statistics, and human revision cost.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The results of MetaGPT on the SoftwareDev dataset show that it achieves an average score of 3.9, surpassing ChatDev's score of 2.1. Additionally, MetaGPT outperforms ChatDev on nearly all metrics, including executability, code statistics, and human revision cost.


In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [ ]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [ ]:
step_output = agent.run_step(task.task_id)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_metagpt
Action Input: {'query': 'agent roles in MetaGPT', 'page_numbers': None}
Observation: In MetaGPT, there are five defined roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer.


In [ ]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 42d9e8c7-1260-4bf1-97dd-09168e8fc483: 1
In MetaGPT, there are five defined roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer.


In [ ]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 42d9e8c7-1260-4bf1-97dd-09168e8fc483: 1


TaskStep(task_id='42d9e8c7-1260-4bf1-97dd-09168e8fc483', step_id='f85c3c09-fa65-4bca-ba1d-1d147dd84601', input=None, step_state={'is_first': False}, next_steps={}, prev_steps={}, is_ready=True)

In [ ]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
Thought: I need to use a tool to help me answer the question about how agents share information in MetaGPT.
Action: vector_tool_metagpt
Action Input: {'query': 'how agents share information in MetaGPT'}
Observation: In MetaGPT, agents share information through a global message pool, where they publish and access messages directly. This allows agents to exchange messages transparently, eliminating the need to inquire about other agents and await their responses. Additionally, a subscription mechanism is used to filter out irrelevant information, enabling agents to receive only task-related information and avoid distractions.


In [ ]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: In MetaGPT, agents share information through a global message pool, where they publish and access messages directly. This allows agents to exchange messages transparently, eliminating the need to inquire about other agents and await their responses. Additionally, a subscription mechanism is used to filter out irrelevant information, enabling agents to receive only task-related information and avoid distractions.
True


In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))

In MetaGPT, agents share information through a global message pool, where they publish and access messages directly. This allows agents to exchange messages transparently, eliminating the need to inquire about other agents and await their responses. Additionally, a subscription mechanism is used to filter out irrelevant information, enabling agents to receive only task-related information and avoid distractions.
